# **2 - ETL**
Também utilizando os dados de viagens de táxi realizadas em New York, agora vamos construir um processo ETL, este será responsável por escrever um output com as seguintes informações e características:

- Qual vendor mais viajou de táxi em cada ano
**Critério de desempate:** quem percorreu o maior percurso, ordem alfabética dos nomes
- Qual a semana de cada ano que mais teve viagens de táxi.
- Quantas viagens o vendor com mais viagens naquele ano fez na semana com mais viagens de táxi no ano.

**O que esperamos:**

- Fundamentos de linguagem de programação e estruturas de dados
- Boas práticas de programação
- Código limpo e simples de entender

**Entrega:**

O entregável desse exercício será um pacote .zip com o código e o output de execução do ETL em formato CSV. Atente-se a deixar descrito um passo a passo de como executar a sua solução.

**Bônus:** Deixe comentários no código explicando o que você estava pensando no momento em que escreveu aquele bloco de código.

## Preparação do Ambiente

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown '1DOvZ-lUlRwyc8jStSSe4Ps0kncHSvhkT'
!gdown '1ilCYiB72T8UPerLiku1c6qdRh94vAUhK'
!gdown '1-UD_8gnTO1UwW-ZQYbW-2WXlwAK7wsl4'
!gdown '10eAuCp7pdUzmBj1SuN_zae3Vo59Wsrfy'
!ls -lt

## Load Data

In [ ]:
import pandas as pd
import glob

In [ ]:
# Use glob to find all JSON files in a directory
json_files = glob.glob('data-nyctaxi-trips-20*.json')

df = pd.concat([pd.read_json(file, lines=True) for file in json_files],
               ignore_index=True)

## Entendendo o Dataset
O código desta sessão eu fiz para entender melhor os datasets, contudo não deixaria isso para um ambiente de produção.

In [ ]:
print("Dataframe:\n{} rows\n{} columns"
      .format(df.shape[0],
              df.shape[1]))

In [ ]:
df.info()

In [ ]:
df.index

In [ ]:
df.head(3)

In [ ]:
df.nunique()

In [ ]:
df.query("vendor_id == 'TS'")

## Preparação de Algumas Variáveis
Esta variáveis serão utilizadas em mais de um lugar.

In [ ]:
# Convert the 'pickup_datetime' column to datetime
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

# Extract the year from the 'pickup_datetime' column
df['year'] = df['pickup_datetime'].dt.isocalendar().year
df['week'] = df['pickup_datetime'].dt.isocalendar().week

## **Qual vendor mais viajou de táxi em cada ano**
### Critério de desempate: quem percorreu o maior percurso, ordem alfabética dos nomes

Para facilitar o entendimento criarei 2 dataframes, um para calcular o total de trips por ano e outro para somar o trip_distance por ano:
- `df_grouped_total_trips` (principal)
- `df_grouped_sum_trips_dist` (critério de desempate)

<br/>

Depois irei fazer um `merge()` para aplicar o critério de desempate no df principal. Por fim, irei retornar o nome e ano de cada vendor 

In [ ]:
# Group the data by year and vendor_id and calculate the total of the trips
# size(): count the number of trips in each group
df_grouped_total_trips = df\
  .groupby(['year', 'vendor_id'])\
  .size()\
  .reset_index(name='counts')

# Group by year and vendor_id and calculate the sum of the trip_distance
df_grouped_sum_trips_dist = df\
  .groupby(['year','vendor_id'])\
  ['trip_distance']\
  .sum()\
  .reset_index(name='trip_distance')

In [ ]:
merged_df = df_grouped_total_trips.merge(df_grouped_sum_trips_dist,
                                         on=['year', 'vendor_id'])

In [ ]:
# Apply tie-breaker
df_vendor_more_trip = merged_df.sort_values(by=['year','counts','trip_distance'], 
                                            ascending=[True,False,False])

# Get the first element by year
df_vendor_more_trip_by_year = df_vendor_more_trip\
  .groupby('year')\
  .first()\
  .reset_index()

In [ ]:
# Prepare df
df_vendor_more_trip_by_year.rename(columns={'vendor_id': 'vendor_more_trip'}, inplace=True)

df_vendor_more_trip_by_year = df_vendor_more_trip_by_year[['year', 'vendor_more_trip']]

## **Qual a semana de cada ano que mais teve viagens de táxi**

In [ ]:
# Group the data by year and week
df_grouped_year_week = df\
  .groupby(['year', 'week'])\
  .size()\
  .reset_index(name='counts')

# Order the data by year, counts and week
df_order_by_year_week = df_grouped_year_week\
  .sort_values(by=['year', 'counts', 'week'],
               ascending=[True, False, True])
  
# Get the first element by year
df_week_more_trips_by_year = df_order_by_year_week\
  .groupby('year')\
  .first()\
  .reset_index()

In [ ]:
# Prepare
df_week_more_trips_by_year.rename(columns={'week': 'week_more_trips'}, inplace=True)

df_week_more_trips_by_year = df_week_more_trips_by_year[['year', 'week_more_trips']]

## **Quantas viagens o vendor com mais viagens naquele ano fez na semana com mais viagens de táxi no ano**
Para resolver este problema vou separar em parte:
1. Obter vendors com mais viagens por ano: `df_vendor_most_trip_by_year`
2. Obter semana com mais trips por ano: `df_week_more_trips_by_year`
3. Obter em cada ano, a semana com mais viagens: `df_week_more_trips`
4. Obter o total de trips pelo vendor com mais trips no ano: `total_trips_week_per_vendor`


In [ ]:
years = df_week_more_trips_by_year['year']
weeks = df_week_more_trips_by_year['week_more_trips']
vendors = df_vendor_more_trip_by_year['vendor_more_trip']

In [ ]:
dict_trips_vendor = {}

for year, week, vendor in zip(years, weeks, vendors):
  df_year = df.loc[df['year'] == year]
  df_week_more_trips = df_year.loc[df['week'] == week]
  total_trips_week_vendor = df_week_more_trips['vendor_id'].loc[df['vendor_id'] == vendor].count()

  dict_trips_vendor[year] = total_trips_week_vendor

In [ ]:
# Prepare df
df_total_trips_vendor = pd.DataFrame(dict_trips_vendor.items(), columns=['year', 'total_trips_per_week'])

In [ ]:
df_result = df_vendor_more_trip_by_year\
  .merge(df_week_more_trips_by_year)\
  .merge(df_total_trips_vendor)

df_result

In [ ]:
df_result.to_csv(path_or_buf = 'output_etl.csv',
                 sep = ',',
                 index = False,
                 encoding = 'utf8')

---

## Observações e Resultado
- Utilizei Pandas para o input, analise e processamento da questão. Caso o volume fosse maior utilizaria Spark.
- Imaginando uma orquestração com este notebook, seria necessário  ter algum bucket ou folder onde possa ser feito a leitura dos arquivos. Sobre o scheduling, poderia executar essa ETL a cada semana pois é o menor valor necessário nas métricas. Apache Airflow pode ser uma boa escolha para orquestração.
- Terminei o notebook limpando o ambiente e exibindo o arquivo csv no terminal.


In [ ]:
!rm -rf data-nyctaxi-trips*.json

In [ ]:
!ls -lt

In [ ]:
!cat 'output_etl.csv'

---